## Fake News Classifier Using Bidirectional LSTM




In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
###Drop Nan Values
df=df.dropna()


In [5]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.3.1'

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [13]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [14]:
messages=X.copy()

In [15]:
messages.reset_index(inplace=True)

In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Smarak
[nltk_data]     Pradhan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
### Dataset Preprocessing
from nltk.stem import LancasterStemmer, WordNetLemmatizer
lemma=WordNetLemmatizer()

def preprocess(doc):
  doc=re.sub(r'\W', ' ',str(doc))
  doc = doc.lower()                 # Converting to lowercase
  cleanr = re.compile('<.*?>')
  doc = re.sub(cleanr, ' ',str(doc))        #Removing HTML tags
  doc = re.sub(r'[?|!|\'|"|#]',r'',str(doc))
  doc = re.sub(r'[.|,|)|(|\|/]',r' ',str(doc))
  doc=re.sub(r'\s+', ' ',str(doc),flags=re.I)
  doc=re.sub(r'^b\s+', ' ',str(doc))
  doc = re.sub(r'\[[0-9]*\]', ' ', doc)
  doc = re.sub(r'\s+', ' ',doc)
  # Removing special characters and digits
  doc = re.sub('[^a-zA-Z]', ' ', doc )
  doc = re.sub(r'\s+', ' ', doc)
  #doc_list = nltk.sent_tokenize(doc)
  stopwords = nltk.corpus.stopwords.words('english')
  #Lemmatization
  tokens=doc.split()
  tokens=[lemma.lemmatize(word) for word in tokens]
  tokens=[word for word in tokens if word not in stopwords]
  
  return tokens
corpus=[]
for i in range(0,len(messages)) :
    tokens=preprocess(messages['title'][i])
    tokens=' '.join(tokens)
    corpus.append(tokens)

In [21]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 


### Embedding Representation

In [22]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1262  265 1737]
 [   0    0    0 ... 3469 4562 2508]
 [   0    0    0 ... 2922 3952 1476]
 ...
 [   0    0    0 ... 2700 2654 2288]
 [   0    0    0 ... 4275 2653 3703]
 [   0    0    0 ... 3854 3760 4557]]


In [23]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0, 3726, 2073,
        874, 4575,  160, 2220, 4580, 1262,  265, 1737])

In [24]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
## Creating model
embedding_vector_features=150
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 150)           750000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               200800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 951,001
Trainable params: 951,001
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
len(embedded_docs),y.shape

(18285, (18285,))

In [29]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [30]:
X_final.shape,y_final.shape

((18285, 30), (18285,))

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [32]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 56ms/step - loss: 0.2989 - accuracy: 0.8626 - val_loss: 0.2006 - val_accuracy: 0.9175
Epoch 2/10
192/192 [==============================] - 10s 50ms/step - loss: 0.1312 - accuracy: 0.9495 - val_loss: 0.2119 - val_accuracy: 0.9130
Epoch 3/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0845 - accuracy: 0.9696 - val_loss: 0.2421 - val_accuracy: 0.9157
Epoch 4/10
192/192 [==============================] - 10s 51ms/step - loss: 0.0475 - accuracy: 0.9845 - val_loss: 0.2966 - val_accuracy: 0.9138
Epoch 5/10
192/192 [==============================] - 10s 50ms/step - loss: 0.0280 - accuracy: 0.9916 - val_loss: 0.3083 - val_accuracy: 0.9145
Epoch 6/10
192/192 [==============================] - 10s 54ms/step - loss: 0.0161 - accuracy: 0.9955 - val_loss: 0.4378 - val_accuracy: 0.9133
Epoch 7/10
192/192 [==============================] - 11s 58ms/step - loss: 0.0104 - accuracy: 0.9968 - val_loss: 0.4414 - val_accuracy:

### Performance Metrics And Accuracy

In [33]:

y_pred1=model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [34]:
from sklearn.metrics import confusion_matrix

In [35]:
confusion_matrix(y_test,y_pred1)

array([[3121,  298],
       [ 282, 2334]], dtype=int64)

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.903893951946976

In [71]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.90      0.92      3419
           1       0.88      0.91      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [37]:
y_pred1

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])